<a href="https://colab.research.google.com/github/Ken89MathCompSci/BERT4Nilm-base/blob/kengoh-corrected-parameters-fridge-and-washer/BERT4NILM-corrected-parameters-fridge-and-washer-18-September-2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ls


BERT4Nilm-base/  sample_data/


In [2]:
pwd

'/content'

In [3]:
import fileinput

trainer_file = '/content/BERT4Nilm-base/trainer.py'

# Use a more robust in-place replacement with python
with fileinput.FileInput(trainer_file, inplace=True, backup='.bak') as file:
    for line in file:
        # Remove the deprecated import
        if 'from torch.autograd.gradcheck import zero_gradients' in line:
            continue # Skip printing this line
        # Replace the deprecated function call with the optimizer's zero_grad() method
        elif 'zero_gradients(self.model.cpu().parameters())' in line:
            # It's important to preserve indentation
            indentation = line[:len(line) - len(line.lstrip())]
            print(f"{indentation}self.optim.zero_grad()");
        else:
            print(line, end='')

print(f"Patched {trainer_file} to fix the 'zero_gradients' ImportError.")

Patched /content/BERT4Nilm-base/trainer.py to fix the 'zero_gradients' ImportError.


In [4]:
with open('/content/BERT4Nilm-base/dataset.py', 'r') as f:
    content = f.read()

# Correct the deprecated pandas '.append' method
# This handles all the broken states we've seen so far
content = content.replace('_entire_data._append', 'entire_data._append') # As seen in the last error
content = content.replace('_entire_data.append', 'entire_data._append') # From the first failed fix
content = content.replace('entire_data.append', 'entire_data._append')  # Original state

# Correct the deprecated '.fillna' method
content = content.replace(".fillna(method='ffill')", ".ffill()")

with open('/content/BERT4Nilm-base/dataset.py', 'w') as f:
    f.write(content)

print("dataset.py has been patched (again, with feeling).")

dataset.py has been patched (again, with feeling).


In [5]:
import torch

if torch.cuda.is_available():
    print("GPU is available.")
    # List all available GPUs and their IDs
    for i in range(torch.cuda.device_count()):
        print(f"GPU ID: {i}, Name: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available. The code will run on CPU.")

GPU is available.
GPU ID: 0, Name: Tesla T4


In [ ]:
!cd /content/BERT4Nilm-base && python train.py